Import necessary packages

In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_google_vertexai import ChatVertexAI
from langchain_anthropic import ChatAnthropic
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage
from typing import List
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
import uuid

import nest_asyncio
import vertexai
nest_asyncio.apply()
vertexai.init(project="simcenter-llm-trial")

#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)


#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

qa_instructions = context_gen("qa_instructions.txt")

a = "Select all faces with radius less than 10"
b = "Select all faces with radius less than 5 and then select their corresponding edges"
c = "Select circular edges with a radius less than 6 and then select their adjacent edges"
d = "Select all faces in the lower half of the model that have a radius less than 10 that are blends"
e = "Select all faces in the xy plane that have a radius greater than 5mm"
f = "Sort all blends by their z position and then select the last two blends"
g = "Select all boltholes in the model"
h = "Select all flanges in the model"
i = "Select all struts in the model and then select their leading edges"
j = "Select all strut fillets in the aftmost section of the model"

#First input to the model, breaks query down and provides geometry definition

class analyzed_query(BaseModel):
    """Identifying whether a geometric feature is present in the database and stating the name of the feature"""
    answer: str= Field(description="YES or NO")
    features: List= Field(description="Name of the identified geometrical feature (Singular)")

class Response(BaseModel):
    """Listing the answer of whether the feature is present in the database and the name of the feature"""
    Response: List[analyzed_query]
    

chat_openai = ChatOpenAI(model="gpt-4", temperature=0.0)
chat_google = ChatVertexAI(model_name="gemini-1.0-pro", temperature=0.0, convert_system_message_to_human=False)
chat_claude = ChatAnthropic(model_name = "claude-3-sonnet-20240229", temperature=0.0) #type: ignore
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)



In [2]:
'''change the human variable to test different inputs'''
human = ("select all leading edges")

def generate_yaml(human:str, chat):
    config = {"metadata": {"conversation_id": str(uuid.uuid4())}}
    parser = PydanticOutputParser(pydantic_object=analyzed_query)
    prompt = ChatPromptTemplate.from_messages([("system", qa_instructions + "Answer the user query. Print the output in Json format according to the instructions.\n{format_instructions}",), ("human", human),]).partial(format_instructions=parser.get_format_instructions())
    chain = prompt | chat | parser
    query_analyzer = chain.invoke({}, config=config)
    answer = query_analyzer.answer

    extracted_features = query_analyzer.features
    print(query_analyzer)
    print()
    loader = DirectoryLoader("RAG")
    raw_docs = loader.load()
    textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = textsplitter.split_documents(raw_docs)
    db = FAISS.from_documents(docs, OpenAIEmbeddings())
    retriever = db.as_retriever()
    

    prompt_for_rag = ChatPromptTemplate.from_template("""You are an assistant that is an expert at RAG (Retrieval Augmented Generation). You have been tasked with looking for the definition of the word provided to you as input and generate a summary of the available information."
                You will use only the context provided to you.                                 
                Context:{context}
                Query_to_db:{query}""")
    definer = ({"context": retriever, "query": RunnablePassthrough()}) | prompt_for_rag | chat | StrOutputParser()
    #Base prompt chain, generates the first yaml version with minimal context

    base_prompt = ChatPromptTemplate.from_messages([("system", system1), few_shot_prompt, MessagesPlaceholder(variable_name="messages")])
    base_chain = base_prompt | chat 
    rag_output = definer.invoke((f"What is the definition of {extracted_features}?"), config=config)

    #print(rag_output)
    #print()

    def chain_yes():
        chain = base_chain | StrOutputParser()
        yaml = chain.invoke({"messages": [human]},config=config)
        #print("RAG not required, generating YAML")
        #print()
       #print(yaml)
        #print()
        return yaml

    def chain_no():
        chain = base_chain | StrOutputParser()
        ai = rag_output
        yaml = chain.invoke({"messages":[ai]}, config=config)
        #print("Getting more information using RAG, then generating YAML")
        #print()
        #print(yaml)
        #print()
        return yaml

    def decision_maker(response_obj: analyzed_query):
        for item in response_obj:
            if item == "YES":
                return chain_yes()
            elif item == "NO":
                return chain_no()

    model_output = []
    for answer in query_analyzer:
        model_output.append(decision_maker(answer))
    #This chain ise used for the reflection prompt, which asks the model to critique and improve on the generated YAML text.
    reflection_prompt = ChatPromptTemplate.from_messages([
            ("system", refsystem),
            few_shot_prompt,
            ("human", rag_output),
            MessagesPlaceholder(variable_name="messages")
    ])

    reflect = reflection_prompt | chat | StrOutputParser()
    message = f"Generate constructive feedback for the YAML Code: \n{model_output[0]}, \nCheck for selection capability and correctness of the code based on the following prompt that was used to generate this code: \n{human}"
    #print(message)
    feedback = reflect.invoke({"messages": [AIMessage(content=message)]},config=config)
    

    final_generate = base_prompt | chat | StrOutputParser()
    output_yaml = final_generate.invoke(
        {"messages": [f"I have some YAML code \n {model_output[0]}" + f"\nThis code needs to be regenerated with the following instructions in mind: \n{feedback}.\n Make sure to output only YAML and nothing else."]}, config=config)
    
    print(remove_code_fences(output_yaml))
    return

In [4]:
human = g
generate_yaml(human, chat_google)

answer='NO' features=['boltholes']


- filter: select * from faces where type like "Cylindrical" and radius < 2
- filter: select * from faces where normal_xangle between 70 and 110 and normal_yangle between 70 and 110
- filter: select * from faces where x < max(x) - 10 or x > min(x) + 10
